# 출력층 설계 (Output layer)

In [1]:
!pip3 install torch torchvision torchaudio

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   --- ------------------------------------ 18.9/216.1 MB 91.7 MB/s eta 0:00:03
   ----- ---------------------------------- 28.3/216.1 MB 94.4 MB/s eta 0:00:02
   ------ --------------------------------- 37.2/216.1 MB 60.6 MB/s eta 0:00:03
   ----------- ---------------------------- 60.3/216.1 MB 72.5 MB/s eta 0:00:03
   ----------- ---------------------------- 64.0/216.1 MB 60.8 MB/s eta 0:00:03
   --------------- ------------------------ 81.8/216.1 MB 65.2 MB/s eta 0:00:03
   ------------------ -------------------- 104.3/216.1 MB 70.8 MB/s eta 0:00:02
   ---------------------- ---------------- 122.9/216.1 MB 72.7 MB/s eta 0:00:02
   ------------------------- ------------- 142.9/216.1 MB 75.4 MB/s eta 0:00:01
   --------------------------- ----------- 152.0/216.1 MB 72.5 MB/s eta 0:00:01
   ------------------------------- ------- 172.2/216.1 MB 74.9 MB/s eta 0:00:01
   --------------------------------- ----- 185.6/

### 소프트 맥스 오버플로우 방지 

In [ ]:
import numpy as np

def softmax(z) :           # 기본모델 : z에 큰값을 넣었을 때 nan 반환
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)   # x = np.array([1000, 1001, 1002]) : [nan nan nan] = 표현가능한 숫자의 범위를 벗어남. overflow / e^x 값이 분모로 들어가기 때문에 계산 불가능임. 


def stable_softmax(z) :    # 오버플로우 방지 
    exp_z = np.exp(z - np.max(z))   # x = np.array([1000, 1001, 1002]) 를 -2, -1, 0으로 반환해서 계산 
    return exp_z / np.sum(exp_z)

x = np.array([1000, 1001, 1002])  
print(softmax(x))   
print(stable_softmax(x))   

# 소프트맥스 함수는 비율로 계산하기 때문에 똑같은 값을 다 빼주면 동일한 비율 반환. 따라서 max 값을 빼줘도 동일한 비율 반환

[nan nan nan]
[0.09003057 0.24472847 0.66524096]


C:\Users\Playdata\AppData\Local\Temp\ipykernel_2828\2980363771.py:4: RuntimeWarning: overflow encountered in exp
  exp_z = np.exp(z)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_2828\2980363771.py:5: RuntimeWarning: invalid value encountered in divide
  return exp_z / np.sum(exp_z)   # x = np.array([1000, 1001, 1002]) : [nan nan nan] = 표현가능한 숫자의 범위를 벗어남. overflow / e^x 값이 분모로 들어가기 때문에 계산 불가능임.


- pytorch 라이브러리 함수 사용

In [ ]:
import torch 
import torch.nn.functional as F   # .nn. : 뉴런네트워크 , 

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)   # softmax는 torch의 tensor 타입에 대해서만 연산함. 또한 dtype=torch.float32 데이터 타입도 float 로 맞춰줘야 계산가능

softmax_output = F.softmax(x)
print(softmax_output)

sigmoid_output = torch.sigmoid(x)     # 시그모이드는 torch 라이브러리에 있음. 
print(sigmoid_output)                 # 0~1사이의 값을 넣어야함. but x 값이 너무 커서 1로 극단적인 값을 반환

tensor([0.0900, 0.2447, 0.6652])
tensor([1., 1., 1.])


C:\Users\Playdata\AppData\Local\Temp\ipykernel_27976\1370867057.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_output = F.softmax(x)


### 손실 함수와 연계

In [ ]:
# 다중클래스 모델을 간단하게 만들어서 학습시켜보자 (흐름만 이해하기). 출력층들이 손실함수와 얼마나 연관이 있는지 알아보자~~

import torch
import torch.nn as nn 
import torch.optim as optim 

# 간단한 다중 클래스 분류 모델 정의 
class SimpleMultiClassModel(nn.Module) :
    def __init__(self):
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5, 3) 

    def forward(self, x) :      # forward 는 순방향으로 진행하는 것 : 예측하는 것 / 오차역전파법에서 사용되는 개념(함수명을 설명하기 위해 나온 이야기~)
        return self.fc(x)  
    
model = SimpleMultiClassModel()
criterion = nn.CrossEntropyLoss()                         # 손실함수 (다음주 예정사항)
optimizer = optim.Adam(model.parameters(), lr=0.01)       # optim = 가중치 최적화 함수 모듈 - model.parameters() 모델파라미터 부여 / 중에서 adam 사용 (다음주 예정사항) / lr=러닝레이트(학습률)

inputs = torch.randn(4, 5)            # 입력값 : 4,5형태의 텐서(배열) 반환 
labels = torch.tensor([0, 2, 1, 0])   # 정답에 대한 라벨 데이터

for _ in range(10):
    preds = model(inputs)                # 순전파
    loss = criterion(preds, labels)      # 손실 계산
    print(loss.item())         

    optimizer.zero_grad()                # 기울기 초기화 (이전 단계에서 계산된 기울기를 0으로 초기화)
    loss.backward()                      # 역전파 (손실에 대한 역전파 수행 - 파라미터에 대한 기울기 계산)
    optimizer.step()                     # 가중치 업데이트 (계산된 기울기를 사용하여 옵티마이저가 모델 파라미터 갱신)

1.5894734859466553
1.5474185943603516
1.5059770345687866
1.4651741981506348
1.4250311851501465
1.3855628967285156
1.3467824459075928
1.3087072372436523
1.2713618278503418
1.2347733974456787
